CREATE TABLE ko_players (
    playerKey INT AUTO_INCREMENT,
    f_name VARCHAR(20),
    l_name VARCHAR(20), 
    playerID INT,
	team VARCHAR(50),
    number INT,
    height VARCHAR(10),
    weight VARCHAR(10),
    birthday DATE,
    MLB_debut_date DATE,
    position VARCHAR(30),
    BT VARCHAR(10),
    PA INT NOT NULL,
    AB INT NOT NULL,
    H INT NOT NULL,
    BB INT NOT NULL,
    RBI INT NOT NULL,
    `AVG` DECIMAL(4,3) NOT NULL,
    SLG DECIMAL(4,3) NOT NULL,
    OBP DECIMAL(4,3) NOT NULL,
    OPS DECIMAL(4,3) NOT NULL,
    PRIMARY KEY (playerKey)
)

populate player data

In [4]:
# Get team ID
import statsapi
import json
 
# Opening JSON file
with open('teams.json', 'r') as openfile:
    # Reading from json file
    t_list = json.load(openfile)

t_info = {}
for team in t_list['teams']:
    res = statsapi.lookup_team(team)[0]

    t_info[res['id']] = res["name"]
print(t_info)

{109: 'Arizona Diamondbacks', 144: 'Atlanta Braves', 110: 'Baltimore Orioles', 111: 'Boston Red Sox', 112: 'Chicago Cubs', 145: 'Chicago White Sox', 113: 'Cincinnati Reds', 114: 'Cleveland Guardians', 115: 'Colorado Rockies', 116: 'Detroit Tigers', 117: 'Houston Astros', 118: 'Kansas City Royals', 108: 'Los Angeles Angels', 119: 'Los Angeles Dodgers', 146: 'Miami Marlins', 158: 'Milwaukee Brewers', 142: 'Minnesota Twins', 121: 'New York Mets', 147: 'New York Yankees', 133: 'Athletics', 143: 'Philadelphia Phillies', 134: 'Pittsburgh Pirates', 135: 'San Diego Padres', 137: 'San Francisco Giants', 136: 'Seattle Mariners', 138: 'St. Louis Cardinals', 139: 'Tampa Bay Rays', 140: 'Texas Rangers', 141: 'Toronto Blue Jays', 120: 'Washington Nationals'}


In [5]:
#Get the players ID and jersey number for roster 
id_num = {}

for t_id in t_info.keys():

    t_roster = statsapi.get('team_roster', {'teamId':t_id})
    for k in t_roster['roster']:
        if k['person']['id'] in id_num.keys():
            print("ID duplicated")
        else:
            try:
                id_num[k['person']['id']] = int(k['jerseyNumber'])
            except:
                id_num[k['person']['id']] = 999

In [ ]:
from player import player
p = player()
p.getAll()
for pl in p.data:
    if pl['position'] == 'P':
        try:
            p_stat = statsapi.player_stat_data(str(pl["playerID"]), 'pitching', 'season')
        except:
            continue
        print(p_stat['stats'][0]['stats']['gamesPlayed'])
    else:
        try:
            p_stat = statsapi.player_stat_data(str(pl["playerID"]), 'hitting', 'season')
        except:
            continue
        print(p_stat['stats'][0]['stats']['plateAppearances'])
        

In [ ]:
#Get all the players
import statsapi
pl = statsapi.get('sports_players',{'season':2024})
n = 0 
pitcher_list = []
batter_list = []
for p in pl['people']:
    if p["id"] in id_num:
        print(p['currentTeam'])
        player = {}
        player["f_name"] = p['firstName']
        player["l_name"] = p['lastName']
        player["playerID"] = p["id"]
        player["team"] = t_info[p['currentTeam']['id']]
        player["number"] = id_num[p["id"]]
        player["height"] = p['height']
        player['weight'] = p['weight']
        player["birthday"] = p["birthDate"]
        try:
            player["MLB_debut_date"] = p["mlbDebutDate"]
        except:
            player["MLB_debut_date"] = None
        player["BT"] = f"{p["batSide"]["code"]}/{p["pitchHand"]["code"]}"
        player["position"] = p["primaryPosition"]['abbreviation']
        # print("id",p["id"])
        if player["position"] == 'P':
            try:
                p_stat = statsapi.player_stat_data(str(p["id"]), 'pitching', 'season')
            except:
                continue
            player['G'] = p_stat['stats'][0]['stats']['gamesPlayed']
            player['GS'] = p_stat['stats'][0]['stats']['gamesStarted']
            player['R'] = p_stat['stats'][0]['stats']['runs']
            player['BB'] = p_stat['stats'][0]['stats']['baseOnBalls']
            player['H'] = p_stat['stats'][0]['stats']['hits']
            player['ERA'] = p_stat['stats'][0]['stats']['era']
            player['IP'] = p_stat['stats'][0]['stats']['inningsPitched']
            player['W'] = p_stat['stats'][0]['stats']['wins']
            player['L'] = p_stat['stats'][0]['stats']['losses']
            player['S'] = p_stat['stats'][0]['stats']['saves']
            player['H'] = p_stat['stats'][0]['stats']['holds']
            player['BS'] = p_stat['stats'][0]['stats']['blownSaves']
            player['ER'] = p_stat['stats'][0]['stats']['earnedRuns']
            pitcher_list.append(player)
        else:
            p_stat = statsapi.player_stat_data(p["id"], 'hitting', 'season')
            player['PA'] = p_stat['stats'][0]['stats']['plateAppearances']
            player['AB'] = p_stat['stats'][0]['stats']['atBats']
            player['H'] = p_stat['stats'][0]['stats']['hits']
            player['BB'] = p_stat['stats'][0]['stats']['baseOnBalls']
            player['RBI'] = p_stat['stats'][0]['stats']['rbi']
            player['AVG'] = p_stat['stats'][0]['stats']['avg']
            player['SLG'] = p_stat['stats'][0]['stats']['slg']
            player['OBP'] = p_stat['stats'][0]['stats']['obp']
            player['OPS'] = p_stat['stats'][0]['stats']['ops']
            batter_list.append(player)

In [16]:
from player import player
p = player()
count = 0
sql = f"INSERT INTO `{p.tn}` ("
for field in p.fields:
    sql += f"`{field}`,"
    count +=1
sql = sql[0:-1] + ') VALUES ('
tokens = ("%s," * count)[0:-1]
sql += tokens + ');'

vals = []
n = 0
for pl in batter_list:
    n += 1
    val = []
    for field in p.fields:
        if field not in pl.keys():
            val.append(0)
        else:
            val.append(pl[field])
    vals.append(val)
    if n % 50 == 0:
        p.cur.executemany(sql,vals)
        vals = []
if len(vals) > 0:
    p.cur.executemany(sql,vals)
# print(sql,vals)
